[structured_output](https://python.langchain.com/v0.1/docs/integrations/chat/ollama_functions/)

# Initialize Model

In [1]:
from package import embedding, llm, agent_llm, url

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(base_url=url, model="mistral")

# Import Lib

In [3]:
import json

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List, Optional

# Structured Output  

Structuring your output into the desired format is one of many techniques you have to master.  
The reasons is when you can format the output as the way you intend to, you open up more possibilites in creating the new way of work with LLM.  

One of many examples of Sturctrued Output done right is:
- use the output as instruction or logic in to another process, mostly done in Agent or LagnGraph scenarios.

## Structured within prompt

In [5]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm who is keen of reading people emotion through the message.
Your job is to classify the message wheter it is positive or negative.
Only extract relavant information from the message.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.

Here is the message: \n\n{message}<|eot_id|>

<|start_header_id|>expert<|end_header_id|>
"""

prompt = PromptTemplate.from_template(prompt_template)

In [6]:
llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I won't talk to you again!!"
response = llm_chain.invoke({"message": message})
print(response)

 Based on the given message, it can be classified as negative. Relevant information extraction:
1. The speaker expresses strong dislike towards the recipient.
2. The speaker states that they cannot endure the recipient's behavior anymore.
3. The speaker threatens to cease communication with the recipient.


In [7]:
message = "Awww this kitten is lovely. I wanna pet her so much. Muah!!"
response = llm_chain.invoke({"message": message})
print(response)

 Based on the given message, I would classify it as positive. The use of words like "lovely," "wanna pet her," and "Muah!!," indicate a strong positive emotion towards the kitten. There is no relevant information to extract besides the emotional classification.


In [8]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message": message})
print(response)

 Based on the given message, I would classify it as positive. The message conveys that the speaker finds the pug to be cute and enjoys the pug's behavior of coming up to them and sleeping on their lap. No specific information is asked to be extracted from this message, so no value will be returned for any attribute.


## Caution
llm.with_structured_output(schema=SentimentExtractor): llama3 and minstral can't use this method due to non-implementation issue  
Hence use this below workaround techniques with parser

## Structured in Pydantic way and later add to prompt

In [31]:
class SentimentExtractor(BaseModel):
    """Sentiment of a message."""
    sentiment:Optional[str] = Field(default=None, description="The sentiment of the message")
    emotion:Optional[str] = Field(default=None, description="The emotion of the message")
    tone:Optional[str] = Field(default=None, description="The tone of the message")
    intent:Optional[str] = Field(default=None, description="The intent of the message")
    action:Optional[str] = Field(default=None, description="The action of the message")
    summary:Optional[str] = Field(default=None, description="The summary of the message")
    message:Optional[str] = Field(default=None, description="The input message")
    entity:Optional[str] = Field(default=None, description="The entity of the message")

# class SentimentExtractor(BaseModel):
#     """Sentiment of a message."""
#     sentiment:str = Field(description="The sentiment of the message")
#     emotion:str = Field(description="The emotion of the message")
#     tone:str = Field(description="The tone of the message")
#     intent:str = Field(description="The intent of the message")
#     action:str = Field(description="The action of the message")
#     summary:str = Field(description="The summary of the message")
#     message:str = Field(description="The input message")

parser = PydanticOutputParser(pydantic_object=SentimentExtractor)

## Normal prompt

In [32]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
"Answer the message.\n{format_instructions}
Remember do not add preamble or explanation.

Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'negative',
 'emotion': 'anger',
 'tone': 'aggressive',
 'intent': 'expressing disappointment and anger towards someone',
 'action': 'ending a relationship or communication',
 'summary': 'The person expresses their frustration and intention to end contact with another person.',
 'message': "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!",
 'entity': ''}

In [33]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'message': 'This pug is crazy, haha. He always comes up to me and sleeps on my lap. What a cutie pie!',
 'sentiment': 'positive',
 'emotion': 'happy',
 'tone': 'playful',
 'intent': 'expressing affection for a pet',
 'action': 'interacting with the pet',
 'summary': 'The sender is describing their interaction with a pug and expressing positive emotions towards it.',
 'entity': 'pug'}

In [34]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
"Answer the message.

Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    # partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"
response = llm_chain.invoke({"message":message})

print(response)

 I'm sorry to hear that the relationship between you and the other person has reached a point where they feel they can no longer communicate with you. It's important to respect their decision and give them space if that's what they need. If there is any way to address the issues that led to this situation, it may be helpful to do so in a calm and respectful manner, but only if both parties are open to it. Otherwise, it might be best to focus on your own well-being and move forward.


In [35]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})

print(response)

 It sounds like you're sharing an affectionate moment with your pug. Pugs are known for their friendly and lovable personalities, so it's not surprising that this one enjoys being close to you. Enjoy the bonding experience with your furry friend!


## Refined prompt

In [44]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm who is keen of reading people through the message.
Only extract relavant information from the message.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.
{format_instructions}
Remember do not add preamble or explanation.
Provide the answer in JSON format.
Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'negative',
 'emotion': 'anger',
 'tone': 'aggressive',
 'intent': 'breakup',
 'action': 'ending a relationship',
 'summary': 'The message expresses anger and intent to end a relationship.',
 'message': "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"}

In [45]:
message = "My brother can't stand you any more. The way you act is extremely unacceptable!! He sweared he won't talk to you again!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'negative',
 'emotion': 'anger',
 'tone': 'aggressive',
 'intent': 'conflict',
 'action': 'refusal to communicate',
 'summary': "The sender's brother has expressed his displeasure with the recipient and has sworn off communication.",
 'message': "My brother can't stand you any more. The way you act is extremely unacceptable!! He sweared he won't talk to you again!"}

In [37]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'message': 'This pug is crazy, haha. He always comes up to me and sleeps on my lap. What a cutie pie!',
 'sentiment': 'positive',
 'emotion': ['happy'],
 'tone': ['playful'],
 'intent': 'Expressing affection for the pug',
 'summary': 'The sender is expressing positive emotions and describes the playful behavior of a pug that sleeps on their lap.',
 'entity': 'pug'}

In [38]:
message = "Oh Jeez! This pizza is out of the world. Where did you get it? I wanna buy some more!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'message': 'Oh Jeez! This pizza is out of the world. Where did you get it? I wanna buy some more!',
 'sentiment': 'positive',
 'emotion': 'excited',
 'tone': 'friendly',
 'intent': 'Purchase request',
 'summary': 'The user expresses excitement about a pizza they have tried and requests to purchase more. They did not specify the location where they got it from.'}

In [39]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm who is keen of reading people through the message.
Only extract relavant information from the message.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.
Remember do not add preamble or explanation.
Provide the answer in JSON format.

Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    # partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I don't wanna talk to you any more"
response = llm_chain.invoke({"message":message})
# json.loads(response)
print(response)

 {
"sentiment": "negative",
"relation_status": "unwanted"
}


In [42]:
json.loads(response)

{'animal_type': 'pug', 'behavior': ['comes up to person', 'sleeps on lap']}

In [40]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})
# json.loads(response)
print(response)

 {
"animal_type": "pug",
"behavior": ["comes up to person", "sleeps on lap"]
}


# Play Ground